In [1]:
import numpy as np # linear algebra
import scipy as scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import dill

import keras as k
import cv2
from tqdm import tqdm


Using Theano backend.


In [2]:
with open('tiffs.pkl', 'rb') as in_strm:
    all_info = dill.load(in_strm)
x_train = all_info[0]
x_test = all_info[1]
y_train = all_info[2]
y_test = all_info[3]
print(x_train.shape)

(35000, 32, 32, 4)


In [3]:
red_dct = scipy.fftpack.dct(x_train[:,:,:,0].astype(np.float64), axis=0)
green_dct = scipy.fftpack.dct(x_train[:,:,:,1].astype(np.float64), axis=0)
blue_dct = scipy.fftpack.dct(x_train[:,:,:,2].astype(np.float64), axis=0)
nir_dct = scipy.fftpack.dct(x_train[:,:,:,3].astype(np.float64), axis=0)
train_dct = np.array([red_dct, green_dct, blue_dct, nir_dct])
train_dct = train_dct.reshape((train_dct.shape[1], train_dct.shape[2], 
                               train_dct.shape[3], train_dct.shape[0]))
print(train_dct.shape)

(35000, 32, 32, 4)


In [ ]:
red_dct = scipy.fftpack.dct(x_test[:,:,:,0].astype(np.float64), axis=0)
green_dct = scipy.fftpack.dct(x_test[:,:,:,1].astype(np.float64), axis=0)
blue_dct = scipy.fftpack.dct(x_test[:,:,:,2].astype(np.float64), axis=0)
nir_dct = scipy.fftpack.dct(x_test[:,:,:,3].astype(np.float64), axis=0)
test_dct = np.array([red_dct, green_dct, blue_dct, nir_dct])
test_dct = test_dct.reshape((test_dct.shape[1], test_dct.shape[2], 
                               test_dct.shape[3], test_dct.shape[0]))
print(test_dct.shape)

In [5]:
full_data_train = np.vstack((x_train, train_dct))
print(full_data_train.shape)

mean = np.mean(full_data_train, axis = 0)
x_train -= mean
train_dct -= mean
std = np.std(full_data_train, axis = 0)
x_train /= std
train_dct /= std

x_test -= mean
x_test /= std

#don't forget the rest later too

(70000, 32, 32, 4)


Extracting features for x_train

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D
from keras.models import Model
from keras.layers.merge import concatenate
from keras.optimizers import Adam

#baseline part
original_input = Input(shape=(32, 32, 4))
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(original_input)
conv2 = Conv2D(64, (3, 3), activation='relu')(conv1)
max1 = MaxPooling2D(pool_size=(2, 2))(conv2)
drop1 = Dropout(0.25)(max1)
flat1 = Flatten()(drop1)
dense1 = Dense(128, activation='relu')(flat1)
drop2 = Dropout(0.5)(dense1)

#mean
dct_input = Input(shape=(32, 32, 4))
conv3 = Conv2D(32, kernel_size=(3, 3), activation='relu')(dct_input)
drop3 = Dropout(0.25)(conv3)
flat2 = Flatten()(drop3)
dense5 = Dense(128, activation='relu')(flat2)
drop4 = Dropout(0.5)(dense5)


combine = concatenate([drop2, drop4])
dense3 = Dense(256, activation='relu')(combine)
dense4 = Dense(17, activation='sigmoid', name='test')(dense3)



model = Model(inputs=[original_input, dct_input], outputs=dense4)

adam = Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                    optimizer=adam,
                    metrics=['accuracy'])


weights = model.get_layer('test').get_weights()
print(len(weights))
print(weights[0])
print(weights[1])



model.fit([x_train, train_dct],
          y_train, batch_size=128, epochs=2, verbose=1)

intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('test').output)
intermediate_output = intermediate_layer_model.predict([x_train, train_means])
print(intermediate_output[0])

from sklearn.metrics import fbeta_score

p_valid = model.predict(x_test, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

2
[[ 0.07742251  0.10007736 -0.1009452  ...,  0.06461939 -0.03712822
   0.09199761]
 [-0.00157036  0.13964671 -0.02464548 ..., -0.02835925 -0.09001972
   0.12961993]
 [ 0.12928566 -0.09925903  0.13807824 ..., -0.11269906 -0.13332392
   0.07795288]
 ..., 
 [ 0.14794046 -0.0615038   0.00622906 ...,  0.12479991  0.12324953
  -0.04725299]
 [ 0.11410967  0.11726999 -0.04934318 ...,  0.03382462 -0.03105235
  -0.02750045]
 [ 0.0738124  -0.00937587  0.02215715 ..., -0.11537861  0.09304836
   0.04384343]]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Epoch 1/2
35000/35000 [==============================] - 267s - loss: 0.2844 - acc: 0.8937   
Epoch 2/2
35000/35000 [==============================] - 257s - loss: 0.2406 - acc: 0.9073   


NameError: name 'train_means' is not defined